In [1]:
# pip install ipython-autotime

In [2]:
import time
import chess
from IPython.display import display, HTML, clear_output
import numpy as np
import pandas as pd

%load_ext autotime

### select a random player 

In [3]:
import random
def random_player(board):
    move = random.choice(list(board.legal_moves))
    return move.uci()

time: 648 µs


### helper functions

In [4]:
def who(player):
    return "White" if player == chess.WHITE else "Black"

time: 875 µs


In [5]:
def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

time: 638 µs


In [6]:
def play_game(player1, player2, visual="svg", pause=0.1):
    """
    playerN1, player2: functions that takes board, return uci move
    visual: "simple" | "svg" | None
    """
    use_svg = (visual == "svg")
    board = chess.Board()
    try:
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                uci = player1(board)
            else:
                uci = player2(board)
            name = who(board.turn)
            board.push_uci(uci)
            board_stop = display_board(board, use_svg)
            html = "<b>Move %s %s, Play '%s':</b><br/>%s" % (
                       len(board.move_stack), name, uci, board_stop)
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        return (None, msg, board)
    result = None
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    if visual is not None:
        print(msg)
#     return (result, msg, board)

time: 3.13 ms


In [7]:
def get_move(prompt):
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

time: 968 µs


In [59]:
def count_pieces(board):
    num_pieces = [0,0]
    
#     for piece in range(6):
    num_pieces[0] += len(board.pieces(chess.PAWN,   chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.BISHOP, chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.KING,   chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.QUEEN,  chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.KNIGHT, chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.ROOK,   chess.WHITE))

    num_pieces[1] += len(board.pieces(chess.PAWN,   chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.BISHOP, chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.KING,   chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.QUEEN,  chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.KNIGHT, chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.ROOK,   chess.BLACK))
                             
    return num_pieces

time: 2.77 ms


In [60]:
def run(player1, player2, iterations):
    df_scoreboard = pd.DataFrame(data={}, columns=['game_result','winner','moves_played', 'remain_w_pieces','remaining_b_pieces'])
    

    for i in range(iterations):
        result = play_game(player1, player2, visual="svg", pause=0)
        result_dict = {'game_result': result[0], 
                       'winner': result[1], 
                       'moves_played': len(result[2].move_stack), 
                       'w_pieces': count_pieces(result[2])[0],
                       'b_pieces': count_pieces(result[2])[1]}
    
#         df_scoreboard.append(result_dict, ignore_index=True)
        df_scoreboard.loc[i] = result_dict
    return df_scoreboard

time: 1.63 ms


### naive evaluation

In [61]:
scoreboard = run(random_player, random_player, 15)

draw: claim
time: 27.6 s


In [62]:
scoreboard

,game_result,winner,moves_played,remain_w_pieces,remaining_b_pieces
0,None,draw: stalemate,261,NaN,NaN
1,None,draw: stalemate,218,NaN,NaN
2,None,draw: insufficient material,257,NaN,NaN
3,None,draw: claim,373,NaN,NaN
4,False,checkmate: Black wins!,404,NaN,NaN
5,None,draw: insufficient material,350,NaN,NaN
6,True,checkmate: White wins!,79,NaN,NaN
7,None,draw: claim,360,NaN,NaN
8,None,draw: stalemate,252,NaN,NaN
9,None,draw: insufficient material,366,NaN,NaN


time: 7.97 ms


In [23]:
def naive_eval(board, move, my_color):
    score = 0
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    return score

time: 1.41 ms


In [24]:
def naive_agent(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = naive_eval(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

time: 1.02 ms


In [27]:
result = play_game(random_player, naive_agent, visual="svg", pause=0)

draw: claim
time: 760 ms


In [28]:
print(result)

(None, 'draw: claim', Board('r1b4q/ppp1ppbp/n1k2p2/8/3P4/8/1K6/8 w - - 21 45'))
time: 952 µs


In [32]:
count_pieces(result[2])

[2, 13]

time: 1.76 ms


### naive evaluation with random heuristic

In [ ]:
def naive_random_heuristic_eval(board, move, my_color):
    score = random.random()
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    # Check global things about the board
    score += 100 if board.is_checkmate() else 0
    return score

In [ ]:
def naive_random_heuristic_agent(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = naive_random_heuristic_eval(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [ ]:
%%time
play_game(random_player, naive_random_heuristic_agent)

### minimax evaluation

In [ ]:
def alphabeta( alpha, beta, depthleft ):
    
    if( depthleft == 0 ):
        return quiesce( alpha, beta )
    for move in board.legal_moves:
        board.push(move)   
        score = -alphabeta( -beta, -alpha, depthleft - 1 )
        board.pop()
        if( score >= beta ):
            return score
        if( score > bestscore ):
            bestscore = score
        if( score > alpha ):
            alpha = score   
    return bestscore



In [ ]:
def minimax_eval(depth, board, move, my_color):
    score = -9999
   
    board.push(move)
   
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    # Check global things about the board
    
    score += 100 if board.is_checkmate() else 0
    return score

In [ ]:
def minimax_agent(board, depth):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()

        move.score = minimax_eval(depth, newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

def minimax(depth, board, is_maximizing):
    if(depth == 0):
        return -evaluation(board)
    
    possibleMoves = board.legal_moves
    if(is_maximizing):
        bestMove = -9999
        for x in possibleMoves:
            move = chess.Move.from_uci(str(x))
            board.push(move)
            bestMove = max(bestMove,minimax(depth - 1, board, not is_maximizing))
            board.pop()
        return bestMove
    else:
        bestMove = 9999
        for x in possibleMoves:
            move = chess.Move.from_uci(str(x))
            board.push(move)
            bestMove = min(bestMove, minimax(depth - 1, board, not is_maximizing))
            board.pop()
        return bestMove